# Final Project

Financial Data Science II

Fall 2021

**Due Tuesday, December 7 at 5:00 PM**

## Guidelines

You should submit both your `.ipynb` file and a `.pdf` version of the notebook via Canvas.

We should be able to take your code, and easily run it. It can load standard Python packages, of course, but it should otherwise be self-contained, not reliant upon any other code.

This is to be completed in your groups. All group members should participate in, and understand, all aspects of the analysis.

**Different groups are not allowed to interact or in any way communicate regarding their analyses.**

We are not looking for a formal "report," but **explain what you are doing** and **state any conclusions in practical terms.** For example, do not just say "The p-value is 0.02." What does that mean in practical terms?

Visualizations will be graded, in part, on their readability. Make fonts sufficiently large. Do not label axes with things such as "CSCORE_B." Use readable names.

You can email me with questions. If it is relevant to the entire class, I will ask you to post it on the Discussion Board. Please do not post questions directly to the Discussion Board regarding this, however.

## The Data

We will continue using the Fannie Mae data set used in the Homeworks 2 and 3. Here we will use a different subset. Just as in Homework 2, it only contains loans for which there is a current unpaid balance, i.e. for which `CURRENT_UPB` is greater than zero.

This data file is available as `project.csv` on Canvas.

## Your Objective

Imagine that you are constructing a simulation model that will generate realistic-looking loans of the type shown in this data set. We will focus here on one particular aspect of this: Simulating credit scores.

Our focus will be on the column `CSCORE_B`.



## Specific Steps

1. There are four missing values in `CSCORE_B`. Fill them in by using the following procedure: Cluster the loans, and then determine which cluster each of the loans with the missing value belong to. Replace the missing credit score value with the average credit score of those observations in the same cluster. Of course, the procedure cannot use `CSCORE_B` as one of the variables used to cluster loans. You can use any clustering procedure of your choice.
2. Visualize the distribution of `CSCORE_B`. Comment on its shape.
3. Find a parametric model that fits to the distribution of `CSCORE_B`. You can consider any distribution you would like, as long as it is parametric. Justify your choice using visualization and AIC. As in any real-world problem, you should not expect the fit to be "perfect." Get the best-fitting parametric distribution that you can. Report the MLE of the parameters for this fit.
4. Visualize how the distribution of `CSCORE_B` varies over different property types, i.e., for different values of `PROP`. Does it seem that the distribution changes across the different values of `PROP`? (Just to be clear, when I say "the distribution changes" I mean any aspect of the distribution changes. For example, going from a Normal(10,2) distribution to a Normal(12,2) is a "change" in distribution.)
5. Is there strong evidence that the distribution of `CSCORE_B` varies across the different values of `PROP`? Test a relevant hypothesis, and report the p-value. This test should be performed using a likelihood ratio test.

## Grading

Each of the five steps listed above will be assessed and scored based on three elements: (1) Appropriate choice of method(s) of analysis and/or visualization, (2) correct implmentation of those method(s), and (3) valid interpretation of the results. Each of these three elements will be given equal weight, and the total points allocated to each step are as follows: Step 1, 18 points. Step 2, 6 points. Step 3, 15 points. Step 4, 6 points. Step 5, 18 points. This is a total of 63 points.

In addition, the notebooks will be assessed based on the overall quality of the visualizations (on a scale of 0 to 6 points) on the overall quality of practical explanations of results (on a scale of 0 to 6 points), and on overall organization of the notebook, i.e., can we find where the different steps are, etc., or is it a mess (on a scale of 0 to 5 points).

Hence, there are total of 80 points.

## Peer Assessment

This is everyone's least favorite part, but I have little choice but to incoporate some component of peer feedback, i.e. assessing how much your groupmates contributed to the work. Without this, I have little basis for avoiding problems where students let the work fall on their groupmates.

Any negative information found through this process will be investigated by the instructor.

I do reserve the right to adjust individual students' grades based on lack of engagement with the project.

The easy way to avoid any problems is to make sure that everyone is engaged and participating. Please advise me of any issues as soon as possible.

## Step 1

Import all the python modules used in the notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gower
import seaborn as sns
from sklearn.manifold import Isomap
from scipy.cluster import hierarchy
from pydiffmap import diffusion_map as dm
from sklearn.cluster import KMeans
from scipy.stats import beta
from scipy.stats import lognorm
from scipy.stats import gamma
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

Helper function to format plots.

In [ ]:
def niceHist(x, bins=None, density=True, figsize=(12,8), label='histogram', xLabel='X', yLabel='Density', grid=True, title=None, labelSize=16, dist=None, fit=None, fitLabel='fit', fitColor='red'):
    plt.figure(figsize=figsize)
    p = plt.hist(x, density=density, label=label, bins=bins)
    if dist is not None:
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        plt.plot(x, dist(x, *fit), label=fitLabel, color=fitColor)
    plt.grid(grid)
    plt.xlabel(xLabel, size=labelSize)
    plt.ylabel(yLabel, size=labelSize)
    plt.suptitle(title, size=labelSize+4)
    plt.legend(prop={'size': labelSize-4})

Load loans data from project.csv file.

In [ ]:
columnNamesShort = (
    "POOL_ID", "LOAN_ID", "ACT_PERIOD", "CHANNEL", "SELLER", 
    "SERVICER", "MASTER_SERVICER", "ORIG_RATE", "CURR_RATE", 
    "ORIG_UPB", "ISSUANCE_UPB", "CURRENT_UPB", "ORIG_TERM", 
    "ORIG_DATE", "FIRST_PAY", "LOAN_AGE", "REM_MONTHS", 
    "ADJ_REM_MONTHS", "MATR_DT", "OLTV", "OCLTV",
    "NUM_BO", "DTI", "CSCORE_B", "CSCORE_C", "FIRST_FLAG", 
    "PURPOSE", "PROP", "NO_UNITS", "OCC_STAT", "STATE", "MSA", 
    "ZIP", "MI_PCT", "PRODUCT", "PPMT_FLG", "IO", 
    "FIRST_PAY_IO", "MNTHS_TO_AMTZ_IO", "DLQ_STATUS", 
    "PMT_HISTORY", "MOD_FLAG", "MI_CANCEL_FLAG", "Zero_Bal_Code",
    "ZB_DTE", "LAST_UPB", "RPRCH_DTE", "CURR_SCHD_PRNCPL", 
    "TOT_SCHD_PRNCPL", "UNSCHD_PRNCPL_CURR", 
    "LAST_PAID_INSTALLMENT_DATE", "FORECLOSURE_DATE",
    "DISPOSITION_DATE", "FORECLOSURE_COSTS", 
    "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
    "ASSET_RECOVERY_COSTS", 
    "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
    "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", "NET_SALES_PROCEEDS",
    "CREDIT_ENHANCEMENT_PROCEEDS", 
    "REPURCHASES_MAKE_WHOLE_PROCEEDS",
    "OTHER_FORECLOSURE_PROCEEDS", "NON_INTEREST_BEARING_UPB", 
    "PRINCIPAL_FORGIVENESS_AMOUNT", "ORIGINAL_LIST_START_DATE", 
    "ORIGINAL_LIST_PRICE", "CURRENT_LIST_START_DATE",
    "CURRENT_LIST_PRICE", "ISSUE_SCOREB", "ISSUE_SCOREC", 
    "CURR_SCOREB", "CURR_SCOREC", "MI_TYPE", "SERV_IND", 
    "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
    "CUMULATIVE_MODIFICATION_LOSS_AMOUNT", 
    "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
    "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS", 
    "HOMEREADY_PROGRAM_INDICATOR",
    "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT", 
    "RELOCATION_MORTGAGE_INDICATOR",
    "ZERO_BALANCE_CODE_CHANGE_DATE", "LOAN_HOLDBACK_INDICATOR", 
    "LOAN_HOLDBACK_EFFECTIVE_DATE","DELINQUENT_ACCRUED_INTEREST", 
    "PROPERTY_INSPECTION_WAIVER_INDICATOR",
    "HIGH_BALANCE_LOAN_INDICATOR", "ARM_5_YR_INDICATOR", 
    "ARM_PRODUCT_TYPE", "MONTHS_UNTIL_FIRST_PAYMENT_RESET", 
    "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
    "INTEREST_RATE_CHANGE_DATE", "PAYMENT_CHANGE_DATE", 
    "ARM_INDEX","ARM_CAP_STRUCTURE", "INITIAL_INTEREST_RATE_CAP", 
    "PERIODIC_INTEREST_RATE_CAP","LIFETIME_INTEREST_RATE_CAP", 
    "MARGIN", "BALLOON_INDICATOR","PLAN_NUMBER", 
    "FORBEARANCE_INDICATOR", 
    "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
    "DEAL_NAME", "RE_PROCS_FLAG", "ADR_TYPE", "ADR_COUNT", 
    "ADR_UPB"
)

columnNamesLong = (
    'Reference Pool ID',	'Loan Identifier',	'Monthly Reporting Period',	'Channel',	'Seller Name',	'Servicer Name',	'Master Servicer',	'Original Interest Rate',	'Current Interest Rate',
    'Original UPB',	'UPB at Issuance',	'Current Actual UPB',	'Original Loan Term',	'Origination Date',	'First Payment Date',	'Loan Age',	'Remaining Months to Legal Maturity',
    'Remaining Months To Maturity',	'Maturity Date',	'Original Loan to Value Ratio (LTV)',	'Original Combined Loan to Value Ratio (CLTV)',	'Number of Borrowers',	'Debt-To-Income (DTI)',
    'Borrower Credit Score at Origination',	'Co-Borrower Credit Score at Origination',	'First Time Home Buyer Indicator',	'Loan Purpose ',	'Property Type',	'Number of Units',
    'Occupancy Status',	'Property State',	'Metropolitan Statistical Area (MSA)',	'Zip Code Short',	'Mortgage Insurance Percentage',	'Amortization Type',	'Prepayment Penalty Indicator',
    'Interest Only Loan Indicator',	'Interest Only First Principal And Interest Payment Date',	'Months to Amortization',	'Current Loan Delinquency Status',	'Loan Payment History',
    'Modification Flag',	'Mortgage Insurance Cancellation Indicator',	'Zero Balance Code',	'Zero Balance Effective Date',	'UPB at the Time of Removal',	'Repurchase Date',
    'Scheduled Principal Current',	'Total Principal Current',	'Unscheduled Principal Current',	'Last Paid Installment Date',	'Foreclosure Date',	'Disposition Date',	'Foreclosure Costs',
    'Property Preservation and Repair Costs',	'Asset Recovery Costs',	'Miscellaneous Holding Expenses and Credits',	'Associated Taxes for Holding Property',	'Net Sales Proceeds',
    'Credit Enhancement Proceeds',	'Repurchase Make Whole Proceeds',	'Other Foreclosure Proceeds',	'Non-Interest Bearing UPB',	'Principal Forgiveness Amount',	'Original List Start Date',
    'Original List Price',	'Current List Start Date',	'Current List Price',	'Borrower Credit Score At Issuance',	'Co-Borrower Credit Score At Issuance',	'Borrower Credit Score Current ',
    'Co-Borrower Credit Score Current',	'Mortgage Insurance Type',	'Servicing Activity Indicator',	'Current Period Modification Loss Amount',	'Cumulative Modification Loss Amount',
    'Current Period Credit Event Net Gain or Loss',	'Cumulative Credit Event Net Gain or Loss',	'HomeReady Program Indicator',	'Foreclosure Principal Write-off Amount',
    'Relocation Mortgage Indicator',	'Zero Balance Code Change Date',	'Loan Holdback Indicator',	'Loan Holdback Effective Date',	'Delinquent Accrued Interest',	'Property Valuation Method ',
    'High Balance Loan Indicator ',	'ARM Initial Fixed-Rate Period  ≤ 5 YR Indicator',	'ARM Product Type',	'Initial Fixed-Rate Period ',	'Interest Rate Adjustment Frequency',
    'Next Interest Rate Adjustment Date',	'Next Payment Change Date',	'Index',	'ARM Cap Structure',	'Initial Interest Rate Cap Up Percent',	'Periodic Interest Rate Cap Up Percent',
    'Lifetime Interest Rate Cap Up Percent',	'Mortgage Margin',	'ARM Balloon Indicator',	'ARM Plan Number',	'Borrower Assistance Plan',	'High Loan to Value (HLTV) Refinance Option Indicator',
    'Deal Name',	'Repurchase Make Whole Proceeds Flag',	'Alternative Delinquency Resolution',	'Alternative Delinquency  Resolution Count',	'Total Deferral Amount '
)

columnNamesMap = dict(zip(columnNamesShort,columnNamesLong))

loandata = pd.read_csv("project.csv", sep='|')
loandata.columns = columnNamesLong

types_from_glossary = [
    'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'DATE',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',
    'DATE',	'DATE',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'DATE',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',
    'NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'DATE',	'NUMERIC',	'ALPHA-NUMERIC',
    'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'DATE',	'NUMERIC',	'DATE',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'DATE',	'DATE',	'DATE',	'NUMERIC',	'NUMERIC',
    'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'DATE',	'NUMERIC',	'DATE',	'NUMERIC',	'NUMERIC',	'NUMERIC ',	'NUMERIC',
    'NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'ALPHA-NUMERIC',	'DATE',	'ALPHA-NUMERIC',	'DATE',	'NUMERIC',
    'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'NUMERIC',	'DATE',	'DATE',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'NUMERIC',	'NUMERIC',
    'NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'ALPHA-NUMERIC',	'NUMERIC',	'NUMERIC'
]

loandata = loandata.dropna(axis=1, how='all')
loandata.head()

Remove columns that have no information in them like loan ID and columns with the same value for all rows.

In [ ]:
loandata2 = loandata.drop(columns=[
    'Reference Pool ID', 'Loan Identifier', 'Monthly Reporting Period', 'Servicer Name', 'Master Servicer', 
    'Metropolitan Statistical Area (MSA)',
    'Zip Code Short','Deal Name', 
    #'UPB at Issuance', 
    'Amortization Type', 'Prepayment Penalty Indicator',
    'Interest Only Loan Indicator', 'Loan Payment History', 'Non-Interest Bearing UPB', 'Principal Forgiveness Amount',
    'Current Period Modification Loss Amount', 'Cumulative Credit Event Net Gain or Loss', 'Foreclosure Principal Write-off Amount',
    'ARM Balloon Indicator', 'High Loan to Value (HLTV) Refinance Option Indicator',
    'Alternative Delinquency  Resolution Count'
]) 

Convert date columns in string MMYYYY format to numeric values (julian dates) to make distance metric more sensible.

In [ ]:
def toJulianDate(df, cols):
    for col in cols:
        datesCol = pd.to_datetime(df[col], format='%m%Y', errors='coerce')
        df[col] = datesCol.apply(pd.Timestamp.to_julian_date)
    return df

loandata2 = toJulianDate(loandata2, ['Origination Date','First Payment Date','Maturity Date'])

# clustering Kmeans with Gower distance

In [ ]:
tmpdata = loandata2.drop(columns=[columnNamesMap['CSCORE_B']])
distMat = gower.gower_matrix(tmpdata.fillna(0))
kminit = KMeans(n_clusters=4, n_init=10)
kmout = kminit.fit(distMat)

In [ ]:
loandata2["kmclust"] = kmout.labels_
loandata2["kmclust"] = loandata2["kmclust"].astype('category').cat.as_unordered()
loandata2['IS_CSCORE_B_NAN'] = loandata2['Borrower Credit Score at Origination'].isnull()

update score based on average from cluster

In [ ]:
tmpdata = loandata2[loandata2['IS_CSCORE_B_NAN']==True][['kmclust', 'IS_CSCORE_B_NAN', columnNamesMap['CSCORE_B']]]
for idx in tmpdata.index.values:
  clusterNum = tmpdata.loc[idx,'kmclust']
  loandata2.loc[idx, columnNamesMap['CSCORE_B']] = loandata2[loandata2['kmclust']==clusterNum][columnNamesMap['CSCORE_B']].mean()



In [ ]:
bla = loandata2[loandata2['IS_CSCORE_B_NAN']==True][['kmclust', 'IS_CSCORE_B_NAN', columnNamesMap['CSCORE_B']]]
bla

## Step 2

In [ ]:
scores = loandata2[columnNamesMap['CSCORE_B']].values
niceHist(scores, bins=np.linspace(500.,900.,41), density=True, label='CSCORE_B', xLabel='CSCORE_B', title='Borrower Credit Score at Origination')

We have a left skewed distribution.

## Step 3

Given that the scores distribution is negatively skewed we can fit a Beta distribution.

In [ ]:
scores1 = scores/1000

In [ ]:
betafit = beta.fit(scores1)
niceHist(
    scores1, bins=np.linspace(0.5,0.9,41), label='CSCORE_B/1000', xLabel='CSCORE_B/1000', 
    title='Borrower Credit Score, Beta function fit', dist=beta.pdf, fit=betafit, 
    fitLabel=f'B($\\alpha$={betafit[0]:.2f}, $\\beta$={betafit[1]:.2f}, l={betafit[2]:.2f}, s={betafit[3]:.2f})'
)


There isn't plenty of choice of negatively skewed distributions so invert the scores and try to fit Lognormal, Gamma and Beta distributions.

In [ ]:

scores2 = 1-scores1
lognormalfit = lognorm.fit(scores2)

niceHist(
    scores2, bins=np.linspace(0.1,0.5,41), label='1 - CSCORE_B/1000', xLabel='1 - CSCORE_B/1000', 
    title='Borrower Credit Score, Lognormal fit', dist=lognorm.pdf, fit=lognormalfit, 
    fitLabel=f'LN($\\sigma$={lognormalfit[0]:.2f}, l={lognormalfit[1]:.2f}, s={lognormalfit[2]:.2f})'
)

In [ ]:
betafit2 = beta.fit(scores2)
niceHist(
    scores2, bins=np.linspace(0.1,0.5,41), label='1 - CSCORE_B/1000', xLabel='1 - CSCORE_B/1000', 
    title='Borrower Credit Score, Beta function fit', dist=beta.pdf, fit=betafit2, 
    fitLabel=f'B($\\alpha$={betafit2[0]:.2f}, $\\beta$={betafit2[1]:.2f}, l={betafit2[2]:.2f}, s={betafit2[3]:.2f})'
)

In [ ]:
gammafit = gamma.fit(scores2)

niceHist(
    scores2, bins=np.linspace(0.1,0.5,41), label='1 - CSCORE_B/1000', xLabel='1 - CSCORE_B/1000', 
    title='Borrower Credit Score, Gamma fit', dist=gamma.pdf, fit=gammafit, 
    fitLabel=f'$\\Gamma$($\\sigma$={gammafit[0]:.2f}, l={gammafit[1]:.2f}, s={gammafit[2]:.2f})'
)

Compute AIC and choose best model:

In [ ]:
aic_lognormal = -2*sum(lognorm.logpdf(scores2,lognormalfit[0], lognormalfit[1], lognormalfit[2]))+6
print(f'Lognormal fit AIC = {aic_lognormal}')
aic_gamma = -2*sum(gamma.logpdf(scores2,gammafit[0], gammafit[1],gammafit[2]))+6
print(f'Gamma fit AIC = {aic_gamma}')
aic_beta = -2*sum(beta.logpdf(scores2,betafit2[0], betafit2[1],betafit2[2], betafit2[3]))+8
print(f'Beta fit AIC = {aic_beta}')

Best fit for Beta!

## Step 4

In [ ]:
loandata2['CSCORE2'] = 1-loandata2[columnNamesMap['CSCORE_B']]/1000
g = sns.FacetGrid(loandata2, row=columnNamesMap['PROP'], height=4, aspect=3)
g.map(sns.kdeplot, 'CSCORE2')

In [ ]:
fig = plt.figure(figsize=(24,12))
ax = fig.add_subplot(121)
ax.set_xlim(0.15, 0.5)
bins = np.linspace(0.1,0.5,36)
fits = {}
for prop in ('SF', 'PU', 'CO'):
    score = loandata2[(loandata2[columnNamesMap['PROP']]==prop)]['CSCORE2']
    plt.hist(score, density=True, label=f'{prop} data', bins=bins, histtype='step')
    fits[prop] = beta.fit(score)    
    x = np.linspace(0.1, 0.5, 100)
    plt.plot(x, beta.pdf(x, *fits[prop]), label=f'{prop} fit: B($\\alpha$={fits[prop][0]:.2f}, $\\beta$={fits[prop][1]:.2f}, l={fits[prop][2]:.2f}, s={fits[prop][3]:.2f})',)
plt.grid(True)
plt.xlabel('1 - CSCORE_B/1000', size=16)
plt.ylabel('Density', size=16)
plt.legend(prop={'size': 12})

ax = fig.add_subplot(122)
ax.set_xlim(0.15, 0.5)
bins = np.linspace(0.1,0.5,36)
fits = {}
for prop in ('CP', 'MH'):
    score = loandata2[(loandata2[columnNamesMap['PROP']]==prop)]['CSCORE2']
    plt.hist(score, density=True, label=f'{prop} data', bins=bins, histtype='step')
    fits[prop] = beta.fit(score)    
    x = np.linspace(0.1, 0.5, 100)
    plt.plot(x, beta.pdf(x, *fits[prop]), label=f'{prop} fit: B($\\alpha$={fits[prop][0]:.2f}, $\\beta$={fits[prop][1]:.2f}, l={fits[prop][2]:.2f}, s={fits[prop][3]:.2f})',)
plt.grid(True)
plt.xlabel('1 - CSCORE_B/1000', size=16)
plt.ylabel('Density', size=16)
plt.suptitle('Distributions of CSCORE_B for different propery types', size=20)
plt.legend(prop={'size': 12})

## Step 5